# 2) Compute features for random parameters

Using the generated random parameters, we next compute responses and features for 3 different feature sets:

- 'bap': features extracted from somatic recording with 3 different steps + a pulse response measured at the soma plus two locations on the apical dendrite (Backpropagating Action Potential)

- 'soma': features extracted from somatic recording with 3 different steps

- 'extra': features extracted from somatic recording with 3 different steps and from mean extracellular action potential

Computed features are saved in the `config/features/` folder for each parameter set and are ready to be used for optimization.

In [ ]:
%load_ext autoreload
%autoreload

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import numpy as np
import os
import json
from pathlib import Path
from scipy.spatial import distance
import MEAutility as mu
import LFPy
from copy import copy
import neuroplotlib as npl
import time

import bluepyopt as bpopt
import bluepyopt.ephys as ephys

%matplotlib notebook

In [ ]:
import l5pc_model
import l5pc_evaluator
import l5pc_plot

In [ ]:
np.random.seed(2308)

In [ ]:
save_features = True

### Define extracellular electrodes

Importantly, if the electrode design changes, new features need to be generated. 

However, a subset of channels can be selected before running the optimization procedure.

In [ ]:
mea_type = 'planar' # 'linear' | 'planar'

In [ ]:
if mea_type == 'linear':
    mea_positions = np.zeros((20, 3))
    mea_positions[:, 2] = 20
    mea_positions[:, 1] = np.linspace(-500, 1000, 20)
    probe = mu.return_mea(info={'pos': list([list(p) for p in mea_positions]), 'center': False, 'plane': 'xy'})
elif mea_type == 'planar':
    #mea_dim = [50, 20]  # n rows x n cols
    mea_dim = [20, 4]
#     mea_pitch = [35, 35]  # rows and cols pitch
    mea_pitch = [70, 35]
    mea_size = 8

    mea_info = {'dim': mea_dim,
                'electrode_name': 'hd-mea',
                'pitch': mea_pitch,
                'shape': 'square',
                'size': 8,
                'type': 'mea',
                'plane': 'xy'}

    probe = mu.return_mea(info=mea_info)
    # Move the MEA out of the neuron plane (yz)
    probe.move([0, 200, 20])
    # Instantiate LFPy electrode object
    # TODO calculate mapping here!!!!
    
electrode = LFPy.RecExtElectrode(probe=probe)
# cell = LFPy.Cell('morphology/C060114A7.asc')
# electrode.calc_mapping(cell)
# del cell
# print(electrode.mapping.shape)

In [ ]:
ax = mu.plot_probe(probe)

### Compute features

Features for the different feature sets ('bap', 'soma', 'extra') are listed in the `config/feature_list.json` file.

Here we loop through the different feature sets and random parameters, compute the corresponding features and save them in a `json` file that will be later used to construct the `CellEvaluator`.


In [ ]:
random_params_file = 'config/params/smart_random.csv'
random_params = pd.read_csv(random_params_file, index_col='index')
random_params

In [ ]:
#TODO improve this: compute all features once and populate different feature_dicts accordingly

In [ ]:
feature_sets = ["extra", "soma", "bap"] # 'soma'/'bap'
channels = 'map'

morphology = ephys.morphologies.NrnFileMorphology('morphology/C060114A7.asc', do_replace_axon=True)
param_configs = json.load(open('config/parameters.json'))
parameters = l5pc_model.define_parameters()
mechanisms = l5pc_model.define_mechanisms()

l5pc_cell = ephys.models.LFPyCellModel('l5pc', 
                                       v_init=-65., 
                                       morph=morphology, 
                                       mechs=mechanisms, 
                                       params=parameters)

param_names = [param.name for param in l5pc_cell.params.values() if not param.frozen]      
responses = {}
features = {}
for feature_set in feature_sets:
    print(f'Feature set {feature_set}')
    responses[feature_set] = []
    features[feature_set] = []

    if feature_set == "extra":
        fitness_protocols = l5pc_evaluator.define_protocols(electrode) 
    else:
        fitness_protocols = l5pc_evaluator.define_protocols() 

    if feature_set == "extra":
        sim = ephys.simulators.LFPySimulator(LFPyCellModel=l5pc_cell, cvode_active=True, electrode=electrode)
    else:
        sim = ephys.simulators.LFPySimulator(LFPyCellModel=l5pc_cell, cvode_active=True)
    
    for i, (index, params) in enumerate(random_params.iterrows()):
        print(f'{i+1} / {len(random_params)}, {index}')   
        t_start = time.time()
        if channels is 'map':
            feature_folder = f'config/features/{index}_{mea_type}_map'
        else:
            feature_folder = f'config/features/{index}'
        feature_file, response, feat_dict = l5pc_evaluator.compute_feature_values(params, l5pc_cell, 
                                                                                  fitness_protocols, sim, 
                                                                                  feature_set=feature_set, 
                                                                                  probe=probe, 
                                                                                  channels=channels,
                                                                                  feature_folder=feature_folder, 
                                                                                  save_to_file=save_features)
        t_stop = time.time()
        print(f"Elapsed time: {t_stop -  t_start}")
        features[feature_set].append(feat_dict)
        responses[feature_set].append(response)
        
        if save_features:
            # save probe yaml in the feature folder
            with (Path(feature_folder) / 'probe.json').open('w') as f:
                json.dump(probe.info, f, indent=4)
            print(f"Saved to {feature_file}")

In [ ]:
pos = features["extra"][0]['Step1']['MEA']['pos_peak_relative'][0]

In [ ]:
l5pc_plot.plot_feature_map(pos, probe)

In [ ]:
eap = l5pc_evaluator.calculate_eap(responses=responses['extra'][0], protocols=fitness_protocols,
                                   protocol_name='Step1')

In [ ]:
eap

In [ ]:
fig = l5pc_plot.plot_multiple_responses(responses['bap'], return_fig=True)

In [ ]:
eaps = []
for i, resp in enumerate(responses['extra']):
    eap = l5pc_evaluator.calculate_eap(resp, protocol_name='Step1', protocols=fitness_protocols)
    eaps.append(eap)

In [ ]:
eaps = np.array(eaps)
eaps.shape

In [ ]:
mu.plot_mea_recording(eaps, probe)